In [1]:
import scipy
import scipy.misc
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import  random
from scipy.io import loadmat
import h5py
import time
import tensorflow as tf
import sys
import pandas as pd

In [2]:
sys.path.insert(1, '/home/Documents/Doutorado/tese/baseline/Channel_Estimation_cGAN/cGAN_python')
from mimo_channels_data_generator2 import RandomChannelMimoDataGenerator

In [3]:
###############################################################################
# Channel data configuration
# fix random seed for reproducibility
seed = 1
np.random.seed(seed)

# training parameters
epochs = 100

# Parameters
global Nt
global Nr
Nt = 64  # num of Rx antennas, will be larger than Nt for uplink massive MIMO
Nr = 8  # num of Tx antennas
# the sample is a measurement of Y values, and their collection composes an example. The channel estimation
min_randomized_snr_db = -1
max_randomized_snr_db = 1

# must be done per example, each one having a matrix of Nr x numSamplesPerExample of complex numbers
numSamplesPerExample = 256  # number of channel uses, input and output pairs
# if wants a gradient calculated many times with same channel
numExamplesWithFixedChannel = 1
numSamplesPerFixedChannel = (
    numExamplesWithFixedChannel * numSamplesPerExample
)  # coherence time
# obs: it may make sense to have the batch size equals the coherence time
batch = 1  # numExamplesWithFixedChannel

num_test_examples = 2000
channel_train_input_file = "mimo_fixed.mat"
print("Reading dataset... ",channel_train_input_file)
method = "manual"

# Generator
training_generator = RandomChannelMimoDataGenerator(
    batch_size=batch,
    Nr=Nr,
    Nt=Nt,
    # num_clusters=num_clusters,
    numSamplesPerFixedChannel=numSamplesPerFixedChannel,
    # numSamplesPerExample=numSamplesPerExample, SNRdB=SNRdB,
    numSamplesPerExample=numSamplesPerExample,
    # method='random')
    method=method,
    file = channel_train_input_file
)
if True:
    training_generator.randomize_SNR = True
    training_generator.min_randomized_snr_db = min_randomized_snr_db
    training_generator.max_randomized_snr_db = max_randomized_snr_db
else:
    training_generator.randomize_SNR = True
    training_generator.SNRdB = 0

##############################################################################
# Model configuration
##############################################################################

training_generator.method = "manual"

Reading dataset...  mimo_fixed.mat


In [4]:
inputs, outputs = training_generator.get_examples(9234)
real_image = outputs.reshape((-1,outputs.shape[1]//2, outputs.shape[2], 2))
input_image = inputs.reshape((-1,inputs.shape[1], inputs.shape[2]//2, 2))

In [19]:
df = pd.DataFrame(data=input_image.reshape((-1,256*8*2)))

In [20]:
df.describe()

,0,1,2,3,4,5,6,7,8,9,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
count,9234.000000,9234.000000,9234.000000,9234.000000,9234.000000,9234.000000,9234.000000,9234.000000,9234.000000,9234.000000,...,9234.000000,9234.000000,9234.000000,9234.000000,9234.000000,9234.000000,9234.000000,9234.000000,9234.000000,9234.000000
mean,-0.014512,-0.019277,-0.024042,0.003249,0.032922,-0.013429,-0.015811,-0.003682,0.055447,0.066710,...,0.169374,0.163526,0.137102,0.122807,0.150747,0.137102,0.162876,0.172190,0.177821,0.202729
std,0.999949,0.999868,0.999765,1.000049,0.999512,0.999964,0.999929,1.000047,0.998516,0.997826,...,0.985605,0.986592,0.990611,0.992484,0.988626,0.990611,0.986700,0.985117,0.984116,0.979288
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
50%,-1.000000,-1.000000,-1.000000,1.000000,1.000000,-1.000000,-1.000000,-1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [22]:
df.nonzero().sum().sum()

AttributeError: 'DataFrame' object has no attribute 'nonzero'